In [1]:
# importing lib 

import pandas as pd
import numpy as np
import seaborn as sna
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score

# Reading the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_original=test.copy()
train_original = train.copy()

In [2]:
# Null value only present in the Employment.Type
train['Employment.Type'].fillna(train['Employment.Type'].mode()[0],inplace=True)
test['Employment.Type'].fillna(test['Employment.Type'].mode()[0],inplace=True)

train.shape,test.shape

((233154, 41), (112392, 40))

In [2]:
# convert years and months into months and store in the same columns :AVERAGE.ACCT.AGE
import re
def cal_month(file,col_name):
    clean=[]
    for each in file[col_name]:  # 'AVERAGE.ACCT.AGE'
        w=re.findall(r'\d+',each)
        month=int(w[0])*12+int(w[1])
        clean.append(month)
    file[col_name]=clean

cal_month(train,'AVERAGE.ACCT.AGE')
cal_month(train,'CREDIT.HISTORY.LENGTH')
cal_month(test,'AVERAGE.ACCT.AGE')
cal_month(test,'CREDIT.HISTORY.LENGTH')

In [4]:
# Treatement of outlier and unproper distribution 
train['disbursed_amount_log']=np.log(train['disbursed_amount'])
test['disbursed_amount_log']=np.log(test['disbursed_amount'])

# similarly for asset_cost
train['asset_cost_log']=np.log(train['asset_cost'])
test['asset_cost_log']=np.log(test['asset_cost'])

In [5]:
def cal_age(file,col_name):
    date=[]
    for each in file[col_name]:   # 'Date.of.Birth'
        w=re.findall(r'\d+',each)
        old_age='19'+w[2]
        age=2018-int(old_age)
        date.append(age)
    file['Age']=date

cal_age(train,'Date.of.Birth')
cal_age(test,'Date.of.Birth')

In [6]:
# train['Employment.Type']=train['Employment.Type'].map({'Self employed':0,'Salaried':1})
# test['Employment.Type']=test['Employment.Type'].map({'Self employed':0,'Salaried':1})

In [16]:
train.head()

,ltv,loan_default,disbursed_amount_log,asset_cost_log,Age,total.no.of.accts,total.active.accts,total.overdue.accts,total.current.balance,total.sanctioned.amount,total.disbursed.amount,total.instal.amt
0,89.55,0,10.831272,10.975071,34,0,0,0,0,0,0,0
1,73.23,1,10.760983,11.090568,33,1,1,1,27600,50200,50200,1991
2,89.63,0,10.883279,11.024513,33,0,0,0,0,0,0,0
3,88.48,1,10.959766,11.099121,25,3,0,0,0,0,0,31
4,88.39,1,10.866242,11.007087,41,0,0,0,0,0,0,0


In [6]:
train['total.no.of.accts']=train['PRI.NO.OF.ACCTS']+train['SEC.NO.OF.ACCTS']
train['total.active.accts']=train['PRI.ACTIVE.ACCTS']+train['SEC.ACTIVE.ACCTS']
train['total.overdue.accts']=train['PRI.OVERDUE.ACCTS']+train['SEC.OVERDUE.ACCTS']
train['total.current.balance']=train['PRI.CURRENT.BALANCE']+train['SEC.CURRENT.BALANCE']
train['total.sanctioned.amount']=train['PRI.SANCTIONED.AMOUNT']+train['SEC.SANCTIONED.AMOUNT']
train['total.disbursed.amount']=train['PRI.DISBURSED.AMOUNT']+train['SEC.DISBURSED.AMOUNT']
train['total.instal.amt']=train['PRIMARY.INSTAL.AMT']+train['SEC.INSTAL.AMT']


test['total.no.of.accts']=test['PRI.NO.OF.ACCTS']+test['SEC.NO.OF.ACCTS']
test['total.active.accts']=test['PRI.ACTIVE.ACCTS']+test['SEC.ACTIVE.ACCTS']
test['total.overdue.accts']=test['PRI.OVERDUE.ACCTS']+test['SEC.OVERDUE.ACCTS']
test['total.current.balance']=test['PRI.CURRENT.BALANCE']+test['SEC.CURRENT.BALANCE']
test['total.sanctioned.amount']=test['PRI.SANCTIONED.AMOUNT']+test['SEC.SANCTIONED.AMOUNT']
test['total.disbursed.amount']=test['PRI.DISBURSED.AMOUNT']+test['SEC.DISBURSED.AMOUNT']
test['total.instal.amt']=test['PRIMARY.INSTAL.AMT']+test['SEC.INSTAL.AMT']

In [7]:
# 'UniqueID','disbursed_amount', 'asset_cost','Date.of.Birth','DisbursalDate','Aadhar_flag', 'PAN_flag'
#        'MobileNo_Avl_Flag','VoterID_flag','Driving_flag', 'Passport_flag','loan_default','PRI.NO.OF.ACCTS')
list_del=['UniqueID','disbursed_amount', 'asset_cost','Date.of.Birth','DisbursalDate','Aadhar_flag', 'PAN_flag',
          'VoterID_flag','Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT','MobileNo_Avl_Flag','Current_pincode_ID',
         'State_ID', 'Employee_code_ID', 'branch_id', 'supplier_id', 'manufacturer_id','NEW.ACCTS.IN.LAST.SIX.MONTHS', 
          'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','Employment.Type','NO.OF_INQUIRIES', 'PERFORM_CNS.SCORE',
          'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH']
for each in list_del:
    train=train.drop(each,axis=1)
for each_ in list_del:
    test=test.drop(each_,axis=1)

In [8]:
# Split the cell
X=train.drop('loan_default',1)
y=train.loan_default


#Creating Dummy variable
X=pd.get_dummies(X)
test=pd.get_dummies(test)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

# Reading submission file
submission=pd.read_csv("sample_submission_24jSKY6.csv")
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size =0.2)

# from sklearn.preprocessing import StandardScaler
# sc_X=StandardScaler()
# x_train=sc_X.fit_transform(x_train)
# x_cv=sc_X.transform(x_cv)

In [32]:
train.columns

Index(['ltv', 'loan_default', 'disbursed_amount_log', 'asset_cost_log', 'Age',
       'total.no.of.accts', 'total.active.accts', 'total.overdue.accts',
       'total.current.balance', 'total.sanctioned.amount',
       'total.disbursed.amount', 'total.instal.amt'],
      dtype='object')

In [10]:
from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(random_state=1,max_depth=16,n_estimators=101)
# classifier.fit(x_train,y_train)
# pred_test = classifier.predict(x_cv)
# score=accuracy_score(y_cv,pred_test)
# print('accuracy_score',score)

/home/arya/.local/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
# Logistic Regression using StratifiedKFold
from sklearn.model_selection import StratifiedKFold


# cross validation logistic model with stratified 5 folds
i=1
kf=StratifiedKFold(n_splits=5,shuffle=True)

for train_index,test_index in kf.split(X,y):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y[train_index],y[test_index]
    classifier = RandomForestClassifier(random_state=1,max_depth=16,n_estimators=101)
    classifier.fit(xtr,ytr)
    pred_test = classifier.predict(xvl)
    score=accuracy_score(yvl,pred_test)
    print('accuracy_score',score)
    i+=1
    pred_test=classifier.predict(test)
    pred=classifier.predict_proba(xvl)[:,1]

submission['loan_default']=pred_test   # filling Loan_Status with predictions
submission['UniqueID']=test_original['UniqueID']   # filling Loan_ID with test Loan_ID

# replacing 0 and 1 with N and Y 
# submission['loan_default'].replace(0, 'N',inplace=True) 
# submission['loan_default'].replace(1, 'Y',inplace=True)

# Converting submission file to .csv format 
pd.DataFrame(submission, columns=['UniqueID','loan_default']).to_csv('random1.csv',index=False)


1 of kfold 5
accuracy_score 0.7828315319951964

2 of kfold 5
accuracy_score 0.7826767600952156

3 of kfold 5
accuracy_score 0.782848319787266

4 of kfold 5
accuracy_score 0.7826292086639502

5 of kfold 5
accuracy_score 0.7830152262491958


In [ ]:
from sklearn.model_selection import GridSearchCV

paramgrid={'max_depth':list(range(14,20,1)),'n_estimators':list(range(90,120,2))}
grid_search=GridSearchCV(estimator=classifier,
                         param_grid=paramgrid,
                         scoring = 'accuracy',
                         cv = 6,
                         n_jobs = -1)

grid_search = grid_search.fit(x_train,y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print(best_accuracy)
print(best_parameters)
